# Ensemble Model Training - Simplified Single-Branch Models

This notebook trains separate simplified CNN models for each feature type and then creates an ensemble.

In [21]:
from datetime import datetime
import os
import json
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.utils import to_categorical, Sequence
from keras.models import Model
from keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Average
)
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from modules.PostgresDBHandler import PostgresDBHandler
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

In [22]:
# Configuration
dbParams = {
    "dbname": "mydatabase",
    "user": "myuser",
    "password": "mypassword",
    "host": "postgres_server",
    "port": "5432",
}

EPOCHS = 200
BATCH_SIZE = 32
KFOLD_SPLITS = 5
FIXED_LENGTH = 128

# Feature types to train models for
FEATURE_TYPES = [
    'mel_spectrogram', 'mfcc', 'chromagram', 'spectral_contrast',
    'tonnetz', 'constant_q', 'cqt', 'stft', 'harmonic_percussive', 'onset_strength'
]

# GPU configuration
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Number of available GPUs: {len(gpus)}")
    except RuntimeError as e:
        print(e)

In [23]:
# Initialize database connection
db = PostgresDBHandler(**dbParams)
db.connect()

# Get instrument mappings
instruments_mappings = db.get_mappings_instruments()
num_classes = len(instruments_mappings)
print(f"Number of instrument classes: {num_classes}")
print("Instruments:", instruments_mappings['name'].tolist())

db.close()

Number of instrument classes: 9
Instruments: ['bass', 'flute', 'sax', 'violin', 'piccolo', 'clarinet', 'cello', 'oboe', 'trumpet']


In [24]:
dbConnect = PostgresDBHandler(**dbParams)
dbConnect.connect()
audioIDs = dbConnect.get_all_unique_audio_ids_in_processed()
processed_data = dbConnect.get_processed_fit_data(audioIDs)

all_processed_data = []
for audio_id in audioIDs:
    features = dbConnect.get_all_feature_types_for_audio(audio_id)
    feature_dict = {f['featureTypeName']: f['featurePath'] for f in features}
    instrumentID = dbConnect.get_audio_file(audio_id)['instrumentID']
    feature_dict['instrumentID'] = instrumentID
    all_processed_data.append(feature_dict)

dbConnect.close()

In [25]:
processed_df = pd.DataFrame(all_processed_data)
processed_df

,mel_spectrogram,mfcc,chromagram,spectral_contrast,tonnetz,constant_q,cqt,stft,harmonic_percussive,onset_strength,instrumentID
0,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/66b83711-61...,ensemble_intermediate_results/chromagram/c816d...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/bfc9e4b6...,ensemble_intermediate_results/constant_q/c450f...,ensemble_intermediate_results/cqt/c13f1ccc-799...,ensemble_intermediate_results/stft/63e5fd31-cd...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/8...,1
1,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/7c328090-f2...,ensemble_intermediate_results/chromagram/d373f...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/40aca69f...,ensemble_intermediate_results/constant_q/979c8...,ensemble_intermediate_results/cqt/876f0ffa-da8...,ensemble_intermediate_results/stft/e41473a0-2d...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/d...,6
2,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/249f6263-cc...,ensemble_intermediate_results/chromagram/6cb04...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/338ed518...,ensemble_intermediate_results/constant_q/75bcd...,ensemble_intermediate_results/cqt/09361522-31b...,ensemble_intermediate_results/stft/4b7447ce-65...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/0...,7
3,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/394aa125-7c...,ensemble_intermediate_results/chromagram/bdbee...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/93063168...,ensemble_intermediate_results/constant_q/90ee9...,ensemble_intermediate_results/cqt/bd4ddecd-7cd...,ensemble_intermediate_results/stft/05921c51-fb...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/c...,9
4,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/9a30d25a-b2...,ensemble_intermediate_results/chromagram/00e98...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/246b1a7f...,ensemble_intermediate_results/constant_q/85d60...,ensemble_intermediate_results/cqt/c8077bd6-4c8...,ensemble_intermediate_results/stft/9582db99-49...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/7...,7
...,...,...,...,...,...,...,...,...,...,...,...
265,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/cbe9db00-8f...,ensemble_intermediate_results/chromagram/f4d0b...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/1fcac6a7...,ensemble_intermediate_results/constant_q/28fe3...,ensemble_intermediate_results/cqt/cff7f70e-a9c...,ensemble_intermediate_results/stft/4cc64164-3c...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/c...,9
266,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/74178be3-83...,ensemble_intermediate_results/chromagram/a913e...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/fe983d4a...,ensemble_intermediate_results/constant_q/74d45...,ensemble_intermediate_results/cqt/0161a92e-ebc...,ensemble_intermediate_results/stft/63890388-9f...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/6...,2
267,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/93441477-46...,ensemble_intermediate_results/chromagram/9d453...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/e56fda80...,ensemble_intermediate_results/constant_

In [26]:
def get_input_shape(feature_type, df):
    feature_path_col = feature_type 
    for path in df[feature_path_col]:
        if isinstance(path, str) and os.path.exists(path):
            arr = np.load(path)
            return arr.shape
    raise ValueError(f"No valid file found for {feature_type}")

In [27]:
class SingleFeatureDataGenerator(Sequence):
    def __init__(self, df, feature_col, batch_size=32, shuffle=True, num_classes=None):
        self.df = df.reset_index(drop=True)
        self.feature_col = feature_col
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_classes = num_classes
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def on_epoch_end(self):
        self.indices = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indices]

        X = []
        y = []

        for _, row in batch_df.iterrows():
            try:
                arr = np.load(row[self.feature_col])
            except Exception as e:
                print(f"Error loading {row[self.feature_col]}: {e}")
                continue
        
            if np.isnan(arr).any() or np.isinf(arr).any():
                raise ValueError(f"Feature file {row[self.feature_col]} contains NaNs or Infs.")
        
            arr = (arr - np.mean(arr)) / (np.std(arr) + 1e-8)
            if arr.ndim == 2:
                arr = np.expand_dims(arr, -1)  # shape: (H, W, 1)
        
            X.append(arr)
            y.append(row['instrumentID'])  # already label-encoded
        
        X = np.array(X)
        y = to_categorical(np.array(y), num_classes=self.num_classes)
        
        return X, y

In [28]:
def create_simple_model(input_shape, num_classes, model_name="simple_cnn"):
    input_layer = Input(shape=(*input_shape, 1), name=f"{model_name}_input")

    x = Conv2D(4, (3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)

    x = Flatten()(x)

    output = Dense(num_classes, activation='softmax', name=f"{model_name}_output")(x)

    model = Model(inputs=input_layer, outputs=output, name=model_name)
    return model

In [29]:
results = {}

for feature_type in tqdm(FEATURE_TYPES, desc="Training features"):
    print(f"\n{'='*40}\nTraining model for {feature_type}\n{'='*40}")

    feature_col = feature_type
    feature_df = processed_df.dropna(subset=[feature_col])
    
    # Global label encoder
    label_encoder = LabelEncoder()
    label_encoder.fit(feature_df['instrumentID'])

    feature_df = feature_df.copy()
    feature_df['instrumentID'] = label_encoder.transform(feature_df['instrumentID'])
    num_classes = len(label_encoder.classes_)
    input_shape = get_input_shape(feature_type, feature_df)


    kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42)
    accuracy_list, loss_list, history_list = [], [], []
    classification_reports, confusion_matrices = [], []

    for fold, (train_idx, test_idx) in enumerate(kf.split(feature_df)):
        print(f"\n--- Fold {fold+1}/{KFOLD_SPLITS} ---")
        train_df = feature_df.iloc[train_idx].reset_index(drop=True)
        test_df = feature_df.iloc[test_idx].reset_index(drop=True)

        train_df, val_df = train_test_split(
            train_df, test_size=0.2, random_state=42, stratify=train_df['instrumentID'])

        # Generators (labels are already encoded)
        train_gen = SingleFeatureDataGenerator(train_df, feature_col, BATCH_SIZE, shuffle=True, num_classes=num_classes)
        val_gen   = SingleFeatureDataGenerator(val_df,   feature_col, BATCH_SIZE, shuffle=False, num_classes=num_classes)
        test_gen  = SingleFeatureDataGenerator(test_df,  feature_col, BATCH_SIZE, shuffle=False, num_classes=num_classes)

        # Model
        model = create_simple_model(input_shape, num_classes, model_name=feature_type)
        model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=[early_stopping])
        history_list.append(history.history)

        # Evaluation
        loss, acc = model.evaluate(test_gen)
        loss_list.append(loss)
        accuracy_list.append(acc)
        print(f"{feature_type} - Fold {fold+1} Test accuracy: {acc:.4f}")

        # Predictions & Reports
        y_pred = model.predict(test_gen)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = []
        for _, labels in test_gen:
            y_true.extend(np.argmax(labels, axis=1))
        y_true = np.array(y_true)

        report = classification_report(y_true, y_pred_classes, output_dict=True)
        classification_reports.append(report)
        conf_matrix = confusion_matrix(y_true, y_pred_classes).tolist()
        confusion_matrices.append(conf_matrix)

        # Save model
        os.makedirs(f"models/{feature_type}", exist_ok=True)
        model.save(f"models/{feature_type}/model_fold{fold+1}.keras")

    # Save results
    results[feature_type] = {
        "accuracy_list": accuracy_list,
        "loss_list": loss_list,
        "histories": history_list,
        "classification_reports": classification_reports,
        "confusion_matrices": confusion_matrices,
    }

    with open(f"models/{feature_type}/results.json", "w") as f:
        json.dump(results[feature_type], f, indent=2)

print("\nAll training complete. Models and results saved in 'models/'")

Training features:   0%|          | 0/10 [00:00<?, ?it/s]


Training model for mel_spectrogram

--- Fold 1/5 ---
Epoch 1/200
6/6 [==============================] - 0s 25ms/step - loss: 48.3109 - accuracy: 0.3314 - val_loss: 13.9829 - val_accuracy: 0.4773
Epoch 2/200
6/6 [==============================] - 0s 11ms/step - loss: 16.4988 - accuracy: 0.6860 - val_loss: 16.4730 - val_accuracy: 0.2727
Epoch 3/200
6/6 [==============================] - 0s 10ms/step - loss: 6.7467 - accuracy: 0.8198 - val_loss: 20.8005 - val_accuracy: 0.3409
Epoch 4/200
6/6 [==============================] - 0s 10ms/step - loss: 1.6321 - accuracy: 0.9128 - val_loss: 12.3063 - val_accuracy: 0.5455
Epoch 5/200
6/6 [==============================] - 0s 10ms/step - loss: 2.3631 - accuracy: 0.9477 - val_loss: 16.4750 - val_accuracy: 0.5682
Epoch 6/200
6/6 [==============================] - 0s 11ms/step - loss: 1.9512 - accuracy: 0.9419 - val_loss: 11.8674 - val_accuracy: 0.6591
Epoch 7/200
6/6 [==============================] - 0s 10ms/step - loss: 0.4771 - accuracy: 0.9767 

/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Training features:  10%|█         


Training model for mfcc

--- Fold 1/5 ---
Epoch 1/200
6/6 [==============================] - 0s 19ms/step - loss: 24.6323 - accuracy: 0.2558 - val_loss: 4.3663 - val_accuracy: 0.1364
Epoch 2/200
6/6 [==============================] - 0s 8ms/step - loss: 14.7431 - accuracy: 0.2791 - val_loss: 11.5862 - val_accuracy: 0.1136
Epoch 3/200
6/6 [==============================] - 0s 8ms/step - loss: 7.2521 - accuracy: 0.5523 - val_loss: 6.2306 - val_accuracy: 0.2727
Epoch 4/200
6/6 [==============================] - 0s 8ms/step - loss: 2.9305 - accuracy: 0.6860 - val_loss: 7.4126 - val_accuracy: 0.1818
Epoch 5/200
6/6 [==============================] - 0s 9ms/step - loss: 2.3534 - accuracy: 0.7849 - val_loss: 10.8907 - val_accuracy: 0.1591
Epoch 6/200
6/6 [==============================] - 0s 8ms/step - loss: 1.9040 - accuracy: 0.7616 - val_loss: 7.3521 - val_accuracy: 0.1591
Epoch 7/200
6/6 [==============================] - 0s 9ms/step - loss: 0.9511 - accuracy: 0.8372 - val_loss: 3.6658 - 

/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

6/6 [==============================] - 0s 19ms/step - loss: 2.9728 - accuracy: 0.1453 - val_loss: 2.3125 - val_accuracy: 0.1364
Epoch 2/200
6/6 [==============================] - 0s 8ms/step - loss: 1.7062 - accuracy: 0.3721 - val_loss: 2.5800 - val_accuracy: 0.1364
Epoch 3/200
6/6 [==============================] - 0s 8ms/step - loss: 1.2545 - accuracy: 0.5407 - val_loss: 2.6825 - val_accuracy: 0.1818
Epoch 4/200
6/6 [==============================] - 0s 8ms/step - loss: 0.9487 - accuracy: 0.6512 - val_loss: 2.6434 - val_accuracy: 0.1591
Epoch 5/200
6/6 [==============================] - 0s 8ms/step - loss: 0.7232 - accuracy: 0.7035 - val_loss: 2.6343 - val_accuracy: 0.1364
Epoch 6/200
6/6 [==============================] - 0s 8ms/step - loss: 0.5388 - accuracy: 0.8488 - val_loss: 2.7315 - val_accuracy: 0.1364
Epoch 7/200
6/6 [==============================] - 0s 8ms/step - loss: 0.4277 - accuracy: 0.9012 - val_loss: 2.7726 - val_accuracy: 0.1591
Epoch 8/200
6/6 [=====================

/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


6/6 [==============================] - 0s 19ms/step - loss: 2.4660 - accuracy: 0.2267 - val_loss: 2.1629 - val_accuracy: 0.1591
Epoch 2/200
6/6 [==============================] - 0s 8ms/step - loss: 1.4355 - accuracy: 0.5233 - val_loss: 2.1530 - val_accuracy: 0.1591
Epoch 3/200
6/6 [==============================] - 0s 8ms/step - loss: 1.0947 - accuracy: 0.6512 - val_loss: 2.1441 - val_accuracy: 0.1591
Epoch 4/200
6/6 [==============================] - 0s 8ms/step - loss: 0.8314 - accuracy: 0.6919 - val_loss: 2.1417 - val_accuracy: 0.1591
Epoch 5/200
6/6 [==============================] - 0s 8ms/step - loss: 0.6141 - accuracy: 0.7907 - val_loss: 2.1300 - val_accuracy: 0.1591
Epoch 6/200
6/6 [==============================] - 0s 8ms/step - loss: 0.4125 - accuracy: 0.9012 - val_loss: 2.2085 - val_accuracy: 0.1591
Epoch 7/200
6/6 [==============================] - 0s 8ms/step - loss: 0.2925 - accuracy: 0.9360 - val_loss: 2.2478 - val_accuracy: 0.1591
Epoch 8/200
6/6 [=====================

/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Training features: 100%|██████████


All training complete. Models and results saved in 'models/'


In [30]:
print("\n" + "="*50)
print("Creating Ensemble Predictions")
print("="*50)

processed_df['instrumentID'] = processed_df['instrumentID'] - 1

ensemble_accuracies = []
ensemble_reports = []
ensemble_conf_matrices = []

for fold in range(KFOLD_SPLITS):
    print(f"\n--- Ensemble Fold {fold + 1}/{KFOLD_SPLITS} ---")
    fold_preds = []
    y_true = None

    for feature_type in FEATURE_TYPES:
        # Load model for this fold
        model_path = f"models/{feature_type}/model_fold{fold+1}.keras"
        if not os.path.exists(model_path):
            print(f"Model not found: {model_path}")
            continue
        model = keras.models.load_model(model_path)

        # Get test data for this fold
        feature_df = processed_df[[feature_type, 'instrumentID']].dropna().reset_index(drop=True)
        kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42)
        train_idx, test_idx = list(kf.split(feature_df))[fold]
        test_df = feature_df.iloc[test_idx].reset_index(drop=True)
        test_gen = SingleFeatureDataGenerator(test_df, feature_type, batch_size=BATCH_SIZE, shuffle=False, num_classes=num_classes)
        preds = model.predict(test_gen, verbose=0)
        fold_preds.append(preds)
        if y_true is None:
            # Get true labels from generator
            y_true = []
            for _, labels in test_gen:
                y_true.extend(np.argmax(labels, axis=1))
            y_true = np.array(y_true)

    if fold_preds:
        ensemble_pred = np.mean(fold_preds, axis=0)
        ensemble_pred_classes = np.argmax(ensemble_pred, axis=1)
        acc = accuracy_score(y_true, ensemble_pred_classes)
        ensemble_accuracies.append(acc)
        print(f"Ensemble Accuracy: {acc:.4f}")
        report = classification_report(y_true, ensemble_pred_classes, output_dict=True)
        ensemble_reports.append(report)
        conf_matrix = confusion_matrix(y_true, ensemble_pred_classes).tolist()
        ensemble_conf_matrices.append(conf_matrix)


Creating Ensemble Predictions

--- Ensemble Fold 1/5 ---
Ensemble Accuracy: 0.8333

--- Ensemble Fold 2/5 ---
Ensemble Accuracy: 0.7222

--- Ensemble Fold 3/5 ---
Ensemble Accuracy: 0.7593

--- Ensemble Fold 4/5 ---
Ensemble Accuracy: 0.8704

--- Ensemble Fold 5/5 ---


/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Ensemble Accuracy: 0.8704


In [31]:
os.makedirs("ensemble_results", exist_ok=True)
date_part = datetime.now().date().__str__().replace('-', '_')
results_path = os.path.join("ensemble_results", f"ensemble_results_{date_part}.json")
ensemble_results = {
    "accuracy_list": ensemble_accuracies,
    "classification_reports": ensemble_reports,
    "confusion_matrices": ensemble_conf_matrices,
}
with open(results_path, "w") as f:
    json.dump(ensemble_results, f, indent=2)
print(f"\nEnsemble results saved to: {results_path}")




Ensemble results saved to: ensemble_results/ensemble_results_2025_07_09.json


In [32]:
for fold in range(KFOLD_SPLITS):
    models_fold = []
    inputs = []
    input_names = []
    for feature_type in FEATURE_TYPES:
        model_path = f"models/{feature_type}/model_fold{fold+1}.keras"
        if not os.path.exists(model_path):
            print(f"Model not found for ensemble: {model_path}")
            continue
        model = keras.models.load_model(model_path)
        models_fold.append(model)
        inp = model.input
        inputs.append(inp)
        input_names.append(feature_type)
    if not models_fold:
        print(f"No models found for fold {fold+1}, skipping ensemble model save.")
        continue
    # If all input shapes are the same, use a single input
    input_shapes = [tuple(inp.shape) for inp in inputs]
    if all(s == input_shapes[0] for s in input_shapes):
        ensemble_input = keras.Input(shape=input_shapes[0][1:], name="ensemble_input")
        model_outputs = [m(ensemble_input) for m in models_fold]
        avg = Average()(model_outputs)
        ensemble_model = Model(inputs=ensemble_input, outputs=avg, name=f"ensemble_model_fold{fold+1}")
    else:
        # Multi-input ensemble
        ensemble_inputs = [keras.Input(shape=inp.shape[1:], name=f"{name}_input") for inp, name in zip(inputs, input_names)]
        model_outputs = [m(inp) for m, inp in zip(models_fold, ensemble_inputs)]
        avg = Average()(model_outputs)
        ensemble_model = Model(inputs=ensemble_inputs, outputs=avg, name=f"ensemble_model_fold{fold+1}")
    # Save the ensemble model
    ensemble_model_path = os.path.join("ensemble_results", f"ensemble_model_fold{fold+1}_{date_part}.keras")
    ensemble_model.save(ensemble_model_path)
    print(f"Saved ensemble Keras model for fold {fold+1} to {ensemble_model_path}")


Saved ensemble Keras model for fold 1 to ensemble_results/ensemble_model_fold1_2025_07_09.keras
Saved ensemble Keras model for fold 2 to ensemble_results/ensemble_model_fold2_2025_07_09.keras
Saved ensemble Keras model for fold 3 to ensemble_results/ensemble_model_fold3_2025_07_09.keras
Saved ensemble Keras model for fold 4 to ensemble_results/ensemble_model_fold4_2025_07_09.keras
Saved ensemble Keras model for fold 5 to ensemble_results/ensemble_model_fold5_2025_07_09.keras


In [33]:
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)

print("\nIndividual Model Performance:")
for feature_type in FEATURE_TYPES:
    if feature_type in results:
        accuracies = results[feature_type]['accuracy_list']
        mean_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)
        print(f"  {feature_type}: {mean_acc:.4f} ± {std_acc:.4f}")

print("\nEnsemble Performance:")
ensemble_mean = np.mean(ensemble_accuracies)
ensemble_std = np.std(ensemble_accuracies)
print(f"  Ensemble: {ensemble_mean:.4f} ± {ensemble_std:.4f}")

# Find best individual model
best_individual = max(
    [(ft, np.mean(results[ft]['accuracy_list'])) for ft in FEATURE_TYPES if ft in results],
    key=lambda x: x[1]
)
improvement = ensemble_mean - best_individual[1]
print(f"\nBest Individual Model: {best_individual[0]} ({best_individual[1]:.4f})")
print(f"Ensemble Improvement: {improvement:.4f} ({improvement*100:.2f}%)")


TRAINING SUMMARY

Individual Model Performance:
  mel_spectrogram: 0.6000 ± 0.1583
  mfcc: 0.7370 ± 0.1999
  chromagram: 0.2556 ± 0.0806
  spectral_contrast: 0.3593 ± 0.1018
  tonnetz: 0.1778 ± 0.0432
  constant_q: 0.7111 ± 0.0912
  cqt: 0.7037 ± 0.0759
  stft: 0.2778 ± 0.0586
  harmonic_percussive: 0.5000 ± 0.1665
  onset_strength: 0.1704 ± 0.0590

Ensemble Performance:
  Ensemble: 0.8111 ± 0.0602

Best Individual Model: mfcc (0.7370)
Ensemble Improvement: 0.0741 (7.41%)
